In [1]:
(require booleans redex/reduction-semantics)
(set-cache-size! 200)

Some possibly helpful laws:

- Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
- - Correlary 1: This holds for `¬(GO ∨ SEL)`
- - Correlary 2: This holds for `¬(GO ∨ RES)`
- Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected. (Note: This might be wrong, if GO is given to a selected and resumed pause... this should not be allowed but im not sure...)


In [2]:
(define/ppl p #:no-check
  ((SEL = p_SEL)
   (K0 = p_K0)
   (Kn = p_Kn)))

(SEL = p_SEL)
(K0 = p_K0)
(Kn = p_Kn)


In [3]:
(define/ppl start #:no-check
  (;; p
   (p_GO = (and GO S))
   (p_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(p_SUSP = SUSP)
   ;(p_KILL = KILL)
   ;; q
   (q_GO = (and GO (not S)))
   (q_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(q_SUSP = SUSP)
   ;(q_KILL = KILL)
   ;; out
   (SEL = (or p_SEL q_SEL))
   (K0 = (or p_K0 q_K0))
   (Kn = (or p_Kn q_Kn))))
        

(p_GO = (and GO S))
(p_RES = RES)
(q_GO = (and GO (not S)))
(q_RES = RES)
(SEL = (or p_SEL q_SEL))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


By [S-consistent] we know that S is true. 

In [4]:
(define/ppl add-S #:no-check
  (replace* start [S true]))

(p_GO = (and GO true))
(p_RES = RES)
(q_GO = (and GO (not true)))
(q_RES = RES)
(SEL = (or p_SEL q_SEL))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


Now, let us case dispatch on the first and future instants.

# Case First Instant

In the first instant, the SEL wires must be false, by [sel-start]. We only need to know about `q_SEL` though.

In [5]:
(define/ppl first-instant #:no-check
  (replace* add-S [q_SEL false]))

(p_GO = (and GO true))
(p_RES = RES)
(q_GO = (and GO (not true)))
(q_RES = RES)
(SEL = (or p_SEL false))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


Now, to convert to TVF

In [6]:
(define/ppl converted #:no-check
  (convert-P first-instant))

((+ p_GO) = (and (+ GO) true))
((- p_GO) = (or (- GO) false))
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = (and (+ GO) false))
((- q_GO) = (or (- GO) true))
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (or (+ p_SEL) false))
((- SEL) = (and (- p_SEL) true))
((+ K0) = (or (+ p_K0) (+ q_K0)))
((- K0) = (and (- p_K0) (- q_K0)))
((+ Kn) = (or (+ p_Kn) (+ q_Kn)))
((- Kn) = (and (- p_Kn) (- q_Kn)))


In [7]:
(define/ppl case1-simplify1
  replace* converted
  [(and (+ GO) true) (+ GO)]
  [(or (- GO) false) (- GO)]
  [(and (+ GO) false) false]
  [(or (- GO) true) true]
  [(or (+ p_SEL) false) (+ p_SEL)]
  [(and (- p_SEL) true) (- p_SEL)])

((+ p_GO) = (+ GO))
((- p_GO) = (- GO))
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = false)
((- q_GO) = true)
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (+ p_SEL))
((- SEL) = (- p_SEL))
((+ K0) = (or (+ p_K0) (+ q_K0)))
((- K0) = (and (- p_K0) (- q_K0)))
((+ Kn) = (or (+ p_Kn) (+ q_Kn)))
((- Kn) = (and (- p_Kn) (- q_Kn)))


Note that `¬( GO ∨ (RES ∧ SEL))` is true for `q` (as `GO` and `SEL` are `false`). Therefore all outputs of `q` are false.

In [8]:
(define/ppl all-false #:no-check
  (replace* case1-simplify1
   [(+ q_K0) false] [(- q_K0) true]
   [(+ q_Kn) false] [(- q_Kn) true]))
   

((+ p_GO) = (+ GO))
((- p_GO) = (- GO))
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = false)
((- q_GO) = true)
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (+ p_SEL))
((- SEL) = (- p_SEL))
((+ K0) = (or (+ p_K0) false))
((- K0) = (and (- p_K0) true))
((+ Kn) = (or (+ p_Kn) false))
((- Kn) = (and (- p_Kn) true))


In [9]:
(define/ppl case1-simplify2
  replace* all-false
  [(or (+ p_K0) false) (+ p_K0)]
  [(and (- p_K0) true) (- p_K0)]
  [(or (+ p_Kn) false) (+ p_Kn)]
  [(and (- p_Kn) true) (- p_Kn)])

((+ p_GO) = (+ GO))
((- p_GO) = (- GO))
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = false)
((- q_GO) = true)
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (+ p_SEL))
((- SEL) = (- p_SEL))
((+ K0) = (+ p_K0))
((- K0) = (- p_K0))
((+ Kn) = (+ p_Kn))
((- Kn) = (- p_Kn))


In [10]:
(assert-same (term case1-simplify2) (term (convert-P p)))
  

# Case Later Instant

By [sel-later] we know that in future instant's `q_SEL` is still false. In addition `q_GO` will also still be false because `S` is true. Thus the same reasoning applies to all instants.

# QED